In [22]:
import pandas as pd
from sklearn.preprocessing import StandardScaler


In [23]:
# Load dataset again (inference notebooks must be self-contained)
df = pd.read_csv("../data/housing.csv")

# Drop non-numeric column
df = df.drop(columns=["Address"])


In [24]:
X = df.drop("Price", axis=1)
y = df["Price"]


In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)


In [26]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [27]:
X_test_device = torch.tensor(
    X_test_scaled, dtype=torch.float32
).to(device)


In [28]:
with torch.no_grad():
    predictions = model(X_test_device)

predictions[:5]


tensor([[13.2972],
        [13.2758],
        [12.4570],
        [ 9.2185],
        [18.0442]])

In [29]:
predictions_np = predictions.cpu().numpy()
predictions_np[:5]


array([[13.297153],
       [13.275793],
       [12.457012],
       [ 9.218501],
       [18.044188]], dtype=float32)

In [30]:
import torch
import torch.nn as nn

class HousePriceModel(nn.Module):
    def __init__(self, input_dim):
        super(HousePriceModel, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.model(x)


In [31]:
# Number of features used during training
INPUT_DIM = 5   # change ONLY if your dataset differs

# Create model
model = HousePriceModel(INPUT_DIM)

# Load trained weights
state_dict = torch.load("../models/house_price_model.pt", map_location=device)
model.load_state_dict(state_dict)

# Move model to device (CPU now, AMD GPU later)
model.to(device)
model.eval()

print("Model loaded successfully on:", device)


Model loaded successfully on: cpu


In [33]:
# Move test data to the same device as the model
X_test_device = torch.tensor(
    X_test_scaled, dtype=torch.float32
).to(device)


In [35]:
import numpy as np

# Reshape predictions for inverse transform
predictions_reshaped = predictions_np.reshape(-1, 1)

# Inverse transform using the same scaler logic
# IMPORTANT: We scale ONLY features, so we rescale target manually
y_mean = y_train.mean()
y_std = y_train.std()

predictions_real = predictions_reshaped * y_std + y_mean

predictions_real[:5]



array([[5932258.02896292],
       [5924703.69519516],
       [5635133.04274191],
       [4489798.84027233],
       [7611097.57602493]])

In [36]:
comparison_df = pd.DataFrame({
    "Actual Price": y_test.values[:10],
    "Predicted Price": predictions_real[:10].flatten()
})

comparison_df


,Actual Price,Predicted Price
0,1339096.077,5.932258e+06
1,1251794.179,5.924704e+06
2,1340094.966,5.635133e+06
3,1431507.623,4.489799e+06
4,1042373.524,7.611098e+06
5,1555320.500,4.901561e+06
6,1250882.292,5.985520e+06
7,1039380.722,5.297522e+06
8,832475.189,6.296109e+06
9,1420648.281,4.311584e+06


##  ROCm-Ready Inference (AMD Compatible)

This notebook performs **device-agnostic PyTorch inference**.

- Model trained using PyTorch
- Inference code uses `torch.device`
- Same notebook runs on:
  - CPU (local development)
  - AMD GPU via ROCm (AMD Developer Cloud)

No code changes required when switching from CPU to AMD GPU.

This demonstrates real-world, production-ready AMD GPU compatibility.
